In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 


In [2]:
print_to_excel = True
print_to_Z = True

In [3]:
# read xlsx files from folder
dir_list = os.listdir(Output_path)
xlsx_list = [x for x in dir_list if '.xlsx' in x and '~' not in x]

# create pivot files - dates
xlsx_split_df = pd.DataFrame({'file_name':xlsx_list})
xlsx_split_df = pd.DataFrame(xlsx_split_df.map(lambda x: x[:-5]).file_name.map(lambda x: x.split('_')).tolist()) # drop '.xlsx' and split bu '_'
xlsx_split_df = xlsx_split_df.sort_values([0]).reset_index(drop=True) # sort by data
xlsx_split_pivot = xlsx_split_df.groupby([1,2,3], dropna=False, as_index=False)[0].agg(list)
xlsx_split_pivot = concat_columns(xlsx_split_pivot, [1,2,3]).drop(columns=[1,2,3])
xlsx_split_pivot.concat_columns = xlsx_split_pivot.concat_columns.str.replace('_nan','')
xlsx_split_pairs = xlsx_split_pivot[xlsx_split_pivot[0].str.len()>1]

# files witch doesn't have a pair
xlsx_split_no_pairs = xlsx_split_pivot[xlsx_split_pivot[0].str.len()<2] 
xlsx_split_no_pairs['file_pre_name'] = xlsx_split_no_pairs[0].str[0] + '_' + xlsx_split_no_pairs['concat_columns']
xlsx_split_no_pairs['file_name'] = xlsx_split_no_pairs['file_pre_name'].apply(lambda x:[y for y in xlsx_list if x in y][0])
no_pairs_list = xlsx_split_no_pairs['file_name']

# create table with files names in 2 columns view
xlsx_split_pairs['first_date'] = xlsx_split_pairs[0].str[0] + '_' + xlsx_split_pairs['concat_columns']
xlsx_split_pairs['last_date'] = xlsx_split_pairs[0].str[1] + '_' + xlsx_split_pairs['concat_columns']
xlsx_split_pairs = xlsx_split_pairs.drop(columns=[0])
cols_list = ['first_date','last_date']
for col in cols_list:
    xlsx_split_pairs[f'{col}_file'] = xlsx_split_pairs[col].apply(lambda x:[y for y in xlsx_list if x in y][0])
xlsx_split_pairs = xlsx_split_pairs.drop(columns=cols_list)
xlsx_split_pairs

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11020\1222752961.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xlsx_split_no_pairs['file_pre_name'] = xlsx_split_no_pairs[0].str[0] + '_' + xlsx_split_no_pairs['concat_columns']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11020\1222752961.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xlsx_split_no_pairs['file_name'] = xlsx_split_no_pairs['file_pre_name'].apply(lambda x:[y for y in xlsx_list if x in y][0])
C:\Users\KlimovaAnnaA\AppData\L

,concat_columns,first_date_file,last_date_file
0,ARAP_manual_map,2024-09-13_ARAP_manual_map.xlsx,2024-10-14_ARAP_manual_map.xlsx
1,EUROCHEM_Deals,2024-09-13_EUROCHEM_Deals.xlsx,2024-10-14_EUROCHEM_Deals.xlsx
2,EUROCHEM_SAP_ARAP,2024-09-13_EUROCHEM_SAP_ARAP.xlsx,2024-10-14_EUROCHEM_SAP_ARAP.xlsx
3,EUROCHEM_SAP_Debt,2024-09-13_EUROCHEM_SAP_Debt.xlsx,2024-10-14_EUROCHEM_SAP_Debt.xlsx
4,Ech_Cash,2024-09-13_Ech_Cash.xlsx,2024-10-14_Ech_Cash.xlsx
7,EuroChem_quantum_Debt,2024-09-20_EuroChem_quantum_Debt.xlsx,2024-10-14_EuroChem_quantum_Debt.xlsx
8,EuroChem_quantum_Deriv,2024-09-20_EuroChem_quantum_Deriv.xlsx,2024-10-14_EuroChem_quantum_Deriv.xlsx
11,SUEK_Cash,2024-09-13_SUEK_Cash.xlsx,2024-10-14_SUEK_Cash.xlsx
12,SUEK_SAP_ARAP,2024-09-13_SUEK_SAP_ARAP.xlsx,2024-10-14_SUEK_SAP_ARAP.xlsx
13,SUEK_SAP_Debt,2024-09-13_SUEK_SAP_Debt.xlsx,2024-10-14_SUEK_SAP_Debt.xlsx


In [4]:
xlsx_split_pairs = xlsx_split_pairs[~xlsx_split_pairs.first_date_file.str.contains('ARAP')].reset_index(drop=True)
# xlsx_split_pairs = xlsx_split_pairs[~xlsx_split_pairs.first_date_file.str.contains('invest_nv')].reset_index(drop=True)
# assert 2==1
xlsx_split_pairs

,concat_columns,first_date_file,last_date_file
0,EUROCHEM_Deals,2024-09-13_EUROCHEM_Deals.xlsx,2024-10-14_EUROCHEM_Deals.xlsx
1,EUROCHEM_SAP_Debt,2024-09-13_EUROCHEM_SAP_Debt.xlsx,2024-10-14_EUROCHEM_SAP_Debt.xlsx
2,Ech_Cash,2024-09-13_Ech_Cash.xlsx,2024-10-14_Ech_Cash.xlsx
3,EuroChem_quantum_Debt,2024-09-20_EuroChem_quantum_Debt.xlsx,2024-10-14_EuroChem_quantum_Debt.xlsx
4,EuroChem_quantum_Deriv,2024-09-20_EuroChem_quantum_Deriv.xlsx,2024-10-14_EuroChem_quantum_Deriv.xlsx
5,SUEK_Cash,2024-09-13_SUEK_Cash.xlsx,2024-10-14_SUEK_Cash.xlsx
6,SUEK_SAP_Debt,2024-09-13_SUEK_SAP_Debt.xlsx,2024-10-14_SUEK_SAP_Debt.xlsx
7,SUEK_quantum_Debt,2024-09-13_SUEK_quantum_Debt.xlsx,2024-09-20_SUEK_quantum_Debt.xlsx


In [5]:
# create df for compaire values
for file in ['first_date_file','last_date_file']:
    compaire_df = pd.DataFrame()
    for counter in range(len(xlsx_split_pairs)):

        file_name = xlsx_split_pairs.loc[counter,file]
        if 'Debt' in file_name:
            file_data = pd.read_excel(Output_path+file_name, sheet_name='Debt')
        elif 'Invest' in file_name:
            file_data = pd.read_excel(Output_path+file_name, sheet_name='Invest')
        else:
            file_data = pd.read_excel(Output_path+file_name)
        file_data_info = pd.DataFrame({'name':file_name,'columns':file_data.columns, 'types':file_data.dtypes, 'col_count':file_data.shape[1], 'str_count':file_data.shape[0], 'null_count':file_data.isna().sum(), 'zero_count':file_data[(file_data==0).all(axis=1)].count(), 'sum_values':file_data.astype(int, errors='ignore').sum()})
        file_data_info_values = file_data_info[(file_data_info['types'].astype(str).str.contains('float'))|(file_data_info['types'].astype(str).str.contains('int'))].reset_index(drop=True)
        
        file_data_info_values['concat_columns'] = xlsx_split_pairs.loc[counter,'concat_columns']
        compaire_df = pd.concat([compaire_df,file_data_info_values],axis=0)
        
        if file == 'first_date_file':
            compaire_df_first = compaire_df
        if file == 'last_date_file':
            compaire_df_last = compaire_df

compaire_df_first,compaire_df_last = compaire_df_first.reset_index(drop=True).sort_values('name'),compaire_df_last.reset_index(drop=True).sort_values('name')
compaire_df_merge = compaire_df_first.merge(compaire_df_last, how='outer', left_on=['concat_columns', 'columns'], right_on=['concat_columns', 'columns'], suffixes=['_1','_2'])
# # compaire_df = compaire_df.drop(columns='merge_on')

In [6]:
compaire_df_merge.head(5)

,name_1,columns,types_1,col_count_1,str_count_1,null_count_1,zero_count_1,sum_values_1,concat_columns,name_2,types_2,col_count_2,str_count_2,null_count_2,zero_count_2,sum_values_2
0,2024-09-13_EUROCHEM_Deals.xlsx,Days,int64,12,12,0,0,3716,EUROCHEM_Deals,2024-10-14_EUROCHEM_Deals.xlsx,int64,12,12,0,0,3356
1,2024-09-13_EUROCHEM_Deals.xlsx,Sum_money,float64,12,12,0,0,-63712276,EUROCHEM_Deals,2024-10-14_EUROCHEM_Deals.xlsx,float64,12,12,0,0,-63712276
2,2024-09-13_EUROCHEM_Deals.xlsx,actualQuantity,int64,12,12,0,0,795245,EUROCHEM_Deals,2024-10-14_EUROCHEM_Deals.xlsx,int64,12,12,0,0,795245
3,2024-09-13_EUROCHEM_SAP_Debt.xlsx,Days,int64,16,165,0,0,289195,EUROCHEM_SAP_Debt,2024-10-14_EUROCHEM_SAP_Debt.xlsx,int64,16,166,0,0,293096
4,2024-09-13_EUROCHEM_SAP_Debt.xlsx,amoutn_outstanding,float64,16,165,0,0,888842785169.810059,EUROCHEM_SAP_Debt,2024-10-14_EUROCHEM_SAP_Debt.xlsx,float64,16,166,0,0,882876460166.960083


In [7]:
compaire_df_work = compaire_df_merge[:].fillna(0)
# compaire values
columns_list_eq = ['types','col_count']
columns_list_mostly_eq = ['null_count','sum_values','str_count','zero_count']
for col in columns_list_eq:
    compaire_df_work[f'{col}_is_equal'] = compaire_df_work[f'{col}_1']==compaire_df_work[f'{col}_2']
for col in columns_list_mostly_eq:
    tolerance = 0.4
    max_value = compaire_df_work[compaire_df_work.columns[compaire_df_work.columns.str.contains(col)]].abs().max(axis=1)
    compaire_df_work[f'{col}_is_equal'] = (abs(compaire_df_work[f'{col}_1']-compaire_df_work[f'{col}_2'])/max_value)<=tolerance
    compaire_df_work.loc[compaire_df_work[f'{col}_1']==compaire_df_work[f'{col}_2'], f'{col}_is_equal'] = True

# choose where False
is_equlal_columns = compaire_df_work.columns[compaire_df_work.columns.str.contains('is_equal')].tolist()
compaire_df_work_false = compaire_df_work[(compaire_df_work[is_equlal_columns]==False).any(axis=1)]
compaire_df_work_false_pivot = compaire_df_work_false.pivot_table(index='concat_columns', values=is_equlal_columns, aggfunc='min').reset_index()

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11020\3970424169.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  compaire_df_work = compaire_df_merge[:].fillna(0)


In [8]:
compaire_df_work_false_pivot['manual_check'] = 'NO'
compaire_df_work_false_pivot['comment'] = np.nan

ARAP:

In [9]:
data_ARAP = pd.read_csv('C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP\\ARAP\\ARAP_compare_file.csv')
# choose date
data_list = data_ARAP.date_script.unique().tolist()
last_date = data_list[-1]
pre_last_date = data_list[-2]
# merge data
merge_data = data_ARAP[data_ARAP.date_script==pre_last_date].merge(data_ARAP[data_ARAP.date_script==last_date], how='outer', on=['holding','CompMap','currencyCode'], suffixes=['_pre', '_last']).fillna(0)
# compaire
col_to_compaire = data_ARAP.columns.tolist()[3:-2]
for item in col_to_compaire:
    col_list = merge_data[merge_data.columns[merge_data.columns.str.contains(item)]].columns.tolist()
    merge_data_compaire = is_approximately_equal_for_cols(df=merge_data, col_1=col_list[0] ,col_2=col_list[1] , tolerance=0.4, print_diff=False, name_col_with_diff=item)
# merge_data_compaire

In [10]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_compaire_files.xlsx'
    if print_to_Z == True:
        Output_file = Output_path + Output_file
    pd.DataFrame(no_pairs_list).to_excel(Output_file, sheet_name='No_pairs', index=False)
    new_list(compaire_df_work_false_pivot, Output_file, 'Files_with_problems')
    new_list(compaire_df_work, Output_file, 'Compaire_files')
    new_list(merge_data_compaire,Output_file,'ARAP')